In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd /kaggle/working/yandex_cup_2023_ml_neuroswipe/src

In [ ]:
import os
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, IterableDataset
from tqdm import tqdm
import numpy as np

from model import SwipeCurveTransformer, get_m1_bigger_model
from tokenizers import CharLevelTokenizerv2, KeyboardTokenizerv1
from dataset import NeuroSwipeDatasetv2
from word_generators import GreedyGenerator
from utils import prepare_batch

In [ ]:
IN_KAGGLE = False
RANDOM_SEED = 12

if IN_KAGGLE:
    DATA_ROOT = "/kaggle/input/neuroswipe-defualt-only-v1"
    MODELS_DIR = ""
else:
    DATA_ROOT = "../data/data_separated_grid"
    MODELS_DIR = "../data/trained_models/m1"

In [ ]:
def init_random_seed(value=42):
    # random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    # torch.backends.cudnn.deterministic = True

In [ ]:
init_random_seed(RANDOM_SEED)

In [ ]:
def get_grid(grid_name: str, grids_path: str) -> dict:
    with open(grids_path, "r", encoding="utf-8") as f:
        return json.load(f)[grid_name]

In [ ]:
MAX_TRAJ_LEN = 299

grid_name = "extra"

grid_name_to_grid_path = os.path.join(DATA_ROOT, "gridname_to_grid.json")
grid_name_to_grid = {grid_name: get_grid(grid_name, grid_name_to_grid_path)}


kb_tokenizer = KeyboardTokenizerv1()
word_char_tokenizer = CharLevelTokenizerv2(os.path.join(DATA_ROOT, "voc.txt"))
keyboard_selection_set = set(kb_tokenizer.i2t)

train_path = os.path.join(DATA_ROOT,
                          "train__extra_only_no_errors__2023_11_01__19_49_14.jsonl")

# In case the jupyter notebook is running in kaggle
# with variables  persistence, I don't want it
# to waste around 20 minutes creating train_dataset.

train_dataset = NeuroSwipeDatasetv2(
        data_path = train_path,
        gridname_to_grid = grid_name_to_grid,
        kb_tokenizer = kb_tokenizer,
        max_traj_len = MAX_TRAJ_LEN,
        word_tokenizer = word_char_tokenizer,
        include_time = False,
        include_velocities = True,
        include_accelerations = True,
        has_target=True,
        has_one_grid_only=True,
        include_grid_name=False,
        keyboard_selection_set=keyboard_selection_set,
        total = 349_172
    )

val_path = os.path.join(DATA_ROOT, "valid__in_train_format__extra_only.jsonl")


val_dataset = NeuroSwipeDatasetv2(
    data_path = val_path,
    gridname_to_grid = grid_name_to_grid,
    kb_tokenizer = kb_tokenizer,
    max_traj_len = MAX_TRAJ_LEN,
    word_tokenizer = word_char_tokenizer,
    include_time = False,
    include_velocities = True,
    include_accelerations = True,
    has_target=True,
    has_one_grid_only=True,
    include_grid_name=False,
    keyboard_selection_set=keyboard_selection_set,
    total = 584
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transformer = get_m1_bigger_model(device)

In [ ]:
def cross_entropy_with_reshape(pred, target, ignore_index=-100, label_smoothing=0.0):
    """
    pred - BatchSize x TargetLen x VocabSize
    target - BatchSize x TargetLen
    """
    pred_flat = pred.view(-1, pred.shape[-1])  # BatchSize*TargetLen x VocabSize
    target_flat = target.reshape(-1)  # BatchSize*TargetLen
    return F.cross_entropy(pred_flat,
                           target_flat,
                           ignore_index=ignore_index,
                           label_smoothing=label_smoothing)

In [ ]:
def lr_scheduler(optimizer):
    return torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                      patience=20,
                                                      factor=0.5,
                                                      verbose=True)

In [ ]:
import traceback
from datetime import datetime
import copy

from typing import Callable


def train_eval_loop(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=10, batch_size=32,
                    device=None, early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000,
                    max_batches_per_epoch_val=1000,
                    data_loader_ctor=DataLoader,
                    optimizer_ctor=None,
                    lr_scheduler_ctor=None,
                    shuffle_train=True,
                    label_smoothing = 0.0,
                    dataloader_workers_n=0,
                    criterion_ignore_index = -100,
                    model_name_postfix = "",
                    model_save_root = ".",
                    prepare_batch: Callable = lambda x, y: (x, y)):
    """
    Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
    :param model: torch.nn.Module - обучаемая модель
    :param train_dataset: torch.utils.data.Dataset - данные для обучения
    :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
    :param criterion: функция потерь для настройки модели
    :param lr: скорость обучения
    :param epoch_n: максимальное количество эпох
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
        отсутствие улучшения модели, чтобы обучение продолжалось.
    :param l2_reg_alpha: коэффициент L2-регуляризации
    :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
    :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
    :param data_loader_ctor: функция для создания объекта, преобразующего датасет в батчи
        (по умолчанию torch.utils.data.DataLoader)
    :return: кортеж из двух элементов:
        - среднее значение функции потерь на валидации на лучшей эпохе
        - лучшая модель
    """
    if device is None:
        device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model.to(device)

    if optimizer_ctor is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
    else:
        optimizer = optimizer_ctor(model.parameters(), lr=lr)

    if lr_scheduler_ctor is not None:
        lr_scheduler = lr_scheduler_ctor(optimizer)
    else:
        lr_scheduler = None

    train_dataloader = data_loader_ctor(train_dataset, batch_size=batch_size, shuffle=shuffle_train,
                                        num_workers=dataloader_workers_n)
    val_dataloader = data_loader_ctor(val_dataset, batch_size=batch_size, shuffle=False,
                                      num_workers=dataloader_workers_n)

    best_val_loss = float('inf')
    best_epoch_i = 0

    best_model_path = "m1_bigger_v2.pt"
    best_model = copy.deepcopy(model)

    if os.path.exists(best_model_path):
        best_model.load_state_dict(torch.load(best_model_path))
        print(f"Загружено состояние модели {best_model_path}")

    for epoch_i in tqdm(range(epoch_n), position = 0):
        try:
            model.train()
            mean_train_loss = 0
            train_batches_n = 0
            for batch_i, (batch_x, batch_y) in tqdm(enumerate(train_dataloader), total = min(max_batches_per_epoch_train, len(train_dataset) // batch_size), position=1, leave = False):
                if batch_i > max_batches_per_epoch_train:
                    break

                batch_x, batch_y = prepare_batch(batch_x, batch_y, device)

                pred = model(*batch_x)
                loss = criterion(pred, batch_y, ignore_index = criterion_ignore_index, label_smoothing=label_smoothing)

                model.zero_grad()
                loss.backward()

                optimizer.step()

                mean_train_loss += float(loss)
                train_batches_n += 1

            mean_train_loss /= train_batches_n
            
            print('Среднее значение функции потерь на обучении', mean_train_loss)



            model.eval()
            mean_val_loss = 0
            val_batches_n = 0

            with torch.no_grad():
                for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
                    if batch_i > max_batches_per_epoch_val:
                        break

                    batch_x, batch_y = prepare_batch(batch_x, batch_y, device)

                    pred = model(*batch_x)
                    loss = criterion(pred, batch_y, ignore_index = criterion_ignore_index, label_smoothing=label_smoothing)

                    mean_val_loss += float(loss)
                    val_batches_n += 1

            mean_val_loss /= val_batches_n
            print('Среднее значение функции потерь на валидации', mean_val_loss)

            if mean_val_loss < best_val_loss:
                best_epoch_i = epoch_i
                best_val_loss = mean_val_loss
                best_model = copy.deepcopy(model)
                torch.save(model.state_dict(), os.path.join(model_save_root, best_model_path))
                cur_time = "{:%Y_%m_%d__%H_%M_%S}".format(datetime.now())
                
                grid_name_to_greedy_generator = {grid_name: GreedyGenerator(model, word_char_tokenizer, device)}
                greedy_predictions = predict_greedy_raw(val_dataset, grid_name_to_greedy_generator)
                greedy_accuracy = sum(greedy_prediction == val_target for greedy_prediction, val_target in zip(greedy_predictions, val_targets)) / len(val_targets)
                
                torch.save(model.state_dict(), os.path.join(model_save_root, f"m1_bigger_v2__{cur_time}__{mean_val_loss:.5f}__greed_acc_{greedy_accuracy:.5f}__{model_name_postfix}.pt"))
                print(f"Greedy accuracy = {greedy_accuracy}")
                print('Новая лучшая модель!')
            elif epoch_i - best_epoch_i > early_stopping_patience:
                print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(
                    early_stopping_patience))
                break

            if lr_scheduler is not None:
                lr_scheduler.step(mean_val_loss)

            print()
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
        except Exception as ex:
            print('Ошибка при обучении: {}\n{}'.format(ex, traceback.format_exc()))
            break

    return best_val_loss, best_model


In [ ]:
from tqdm.notebook import tqdm


In [ ]:
def truncate_padding(seq, mask):
    max_curve_len = int(torch.max(torch.sum(~mask, dim = 1)))
    seq = seq[:, :max_curve_len]
    mask = mask[:, :max_curve_len]
    return seq, mask

def prepare_batch_with_pad_truncation(x, y, device):
    (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq = x, y

    xyt, traj_pad_mask = truncate_padding(xyt, traj_pad_mask)
    kb_tokens, traj_pad_mask = truncate_padding(kb_tokens, traj_pad_mask)
#     dec_in_char_seq, word_pad_mask = truncate_padding(kb_tokens, traj_pad_mask)
#     dec_out_char_seq, word_pad_mask = truncate_padding(kb_tokens, traj_pad_mask)

    # print(max_curve_len)

    xyt = xyt.transpose_(0, 1).to(device)  # (curves_seq_len, batch_size, n_coord_feats)
    kb_tokens = kb_tokens.transpose_(0, 1).to(device) # (curves_seq_len, batch_size)
    dec_in_char_seq = dec_in_char_seq.transpose_(0, 1).to(device)  # (chars_seq_len - 1, batch_size)
    dec_out_char_seq = dec_out_char_seq.transpose_(0, 1).to(device)  # (chars_seq_len - 1, batch_size)

    traj_pad_mask = traj_pad_mask.to(device)  # (batch_size, max_curve_len)
    # traj_pad_mask = torch.zeros_like(kb_tokens, dtype = torch.bool).transpose_(0, 1).to(device)
    word_pad_mask = word_pad_mask.to(device)  # (batch_size, chars_seq_len - 1)

    return (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq

prepare_batch = prepare_batch_with_pad_truncation

In [ ]:
from typing import List

def predict_greedy_raw(dataset,
                       grid_name_to_greedy_generator,
                      ) -> List[List[str]]:
    """
    Creates predictions using greedy generation.
    
    Arguments:
    ----------
    dataset: NeuroSwipeDatasetv2
    grid_name_to_greedy_generator: dict
        Dict mapping grid names to GreedyGenerator objects.
    """
    preds = [None] * len(dataset)

    for data in tqdm(enumerate(dataset), total=len(dataset)):
#         i, ((xyt, kb_tokens, _, traj_pad_mask, _), _, grid_name) = data
        i, ((xyt, kb_tokens, _, traj_pad_mask, _), _) = data

        pred = grid_name_to_greedy_generator[grid_name](xyt, kb_tokens, traj_pad_mask)
        pred = pred.removeprefix("<sos>")
#         preds[i] = [pred]
        preds[i] = pred

    return preds

In [ ]:
from typing import List
def get_targets(dataset: NeuroSwipeDatasetv2) -> List[str]:
    targets = []
    for (_, _, _, _, word_pad_mask), target_tokens in dataset:
        target_len = int(torch.sum(~word_pad_mask)) - 1
        target = word_char_tokenizer.decode(target_tokens[:target_len])
        targets.append(target)
    return targets


In [ ]:
val_targets = get_targets(val_dataset)

In [ ]:
transformer.load_state_dict(
    torch.load("/kaggle/working/m1_bigger_v2__2023_11_12__15_09_14__0.13099__greed_acc_0.85939__default_l2_0_ls0_switch_2.pt",
              map_location = device))

In [ ]:
grid_name_to_greedy_generator = {grid_name: GreedyGenerator(transformer, word_char_tokenizer, device)}

In [ ]:
greedy_predictions = predict_greedy_raw(val_dataset, grid_name_to_greedy_generator)

In [ ]:
sum(greedy_prediction == val_target for greedy_prediction, val_target in zip(greedy_predictions, val_targets)) / len(val_targets)

In [ ]:
l2_reg_alpha =  0 #5e-5
label_smoothing=  0 #0.045

best_val_loss, best_model = train_eval_loop(
    transformer, train_dataset, val_dataset, cross_entropy_with_reshape,
    lr=1e-4, epoch_n=10000, batch_size=320,
    device=device, early_stopping_patience=10, l2_reg_alpha=l2_reg_alpha,
    max_batches_per_epoch_train=2000,
    max_batches_per_epoch_val=1000,
    data_loader_ctor=DataLoader,
    optimizer_ctor=None,
    lr_scheduler_ctor=lr_scheduler,
    shuffle_train=True,
    dataloader_workers_n=0,
    criterion_ignore_index = word_char_tokenizer.char_to_idx['<pad>'],
    model_name_postfix = f'{grid_name}_l2_{l2_reg_alpha}_ls{label_smoothing}_switch_2',
    prepare_batch=prepare_batch,
    model_save_root = "../..",
    label_smoothing=label_smoothing
)

In [ ]:
# First epoch: 

# Среднее значение функции потерь на обучении 1.2045975528854778
# Среднее значение функции потерь на валидации 0.5010274122158687

```
Среднее значение функции потерь на обучении 1.2045975528854778
Среднее значение функции потерь на валидации 0.5010274122158687
Новая лучшая модель!

2001/? [11:46<00:00, 2.94it/s]
Среднее значение функции потерь на обучении 0.9051285277063521
Среднее значение функции потерь на валидации 0.4248993237813314
Новая лучшая модель!

2001/? [11:43<00:00, 2.76it/s]
Среднее значение функции потерь на обучении 0.8538736782331338
Среднее значение функции потерь на валидации 0.3928388337294261
Новая лучшая модель!

2001/? [11:43<00:00, 2.56it/s]
Среднее значение функции потерь на обучении 0.8256472818914621
Среднее значение функции потерь на валидации 0.3676854441563288
Новая лучшая модель!

2001/? [11:40<00:00, 2.64it/s]
Среднее значение функции потерь на обучении 0.8055656312823832
Среднее значение функции потерь на валидации 0.35403793156147
Новая лучшая модель!

2001/? [11:42<00:00, 2.82it/s]
Среднее значение функции потерь на обучении 0.791520650925367
Среднее значение функции потерь на валидации 0.34193819264570874
Новая лучшая модель!

2001/? [11:41<00:00, 3.06it/s]
Среднее значение функции потерь на обучении 0.7799409364593559
Среднее значение функции потерь на валидации 0.3324617197116216
Новая лучшая модель!

2001/? [11:44<00:00, 2.47it/s]
Среднее значение функции потерь на обучении 0.7715481882807852
Среднее значение функции потерь на валидации 0.3211148182551066
Новая лучшая модель!

2001/? [11:41<00:00, 3.06it/s]
Среднее значение функции потерь на обучении 0.7643214609550274
Среднее значение функции потерь на валидации 0.31518318951129914
Новая лучшая модель!

2001/? [11:41<00:00, 2.85it/s]
Среднее значение функции потерь на обучении 0.7581036963324616
Среднее значение функции потерь на валидации 0.3094484398762385
Новая лучшая модель!

2001/? [11:46<00:00, 2.62it/s]
Среднее значение функции потерь на обучении 0.7524177488358482
Среднее значение функции потерь на валидации 0.3030394206444422
Новая лучшая модель!

2001/? [11:45<00:00, 2.86it/s]
Среднее значение функции потерь на обучении 0.7466873974576108
Среднее значение функции потерь на валидации 0.2996497412522634
Новая лучшая модель!

2001/? [11:45<00:00, 2.80it/s]
Среднее значение функции потерь на обучении 0.7439289238975979
Среднее значение функции потерь на валидации 0.29270503520965574
Новая лучшая модель!

2001/? [11:44<00:00, 2.80it/s]
Среднее значение функции потерь на обучении 0.739722256002755
Среднее значение функции потерь на валидации 0.2876604378223419
Новая лучшая модель!

2001/? [11:43<00:00, 2.66it/s]
Среднее значение функции потерь на обучении 0.7350960545751942
Среднее значение функции потерь на валидации 0.28610232720772427
Новая лучшая модель!

2001/? [11:42<00:00, 2.75it/s]
Среднее значение функции потерь на обучении 0.7321513988327111
Среднее значение функции потерь на валидации 0.28779847621917726

2001/? [11:44<00:00, 2.98it/s]
Среднее значение функции потерь на обучении 0.7278626844145428
Среднее значение функции потерь на валидации 0.2804383928577105
Новая лучшая модель!

2001/? [11:44<00:00, 2.64it/s]
Среднее значение функции потерь на обучении 0.7260185109860059
Среднее значение функции потерь на валидации 0.2751611083745956
Новая лучшая модель!

2001/? [11:43<00:00, 2.92it/s]
Среднее значение функции потерь на обучении 0.7236041619681168
Среднее значение функции потерь на валидации 0.27163358430067697
Новая лучшая модель!

2001/? [11:47<00:00, 2.97it/s]
Среднее значение функции потерь на обучении 0.7213456676281553
Среднее значение функции потерь на валидации 0.2692498445510864
Новая лучшая модель!

2001/? [11:52<00:00, 2.88it/s]
Среднее значение функции потерь на обучении 0.7186731718171543
Среднее значение функции потерь на валидации 0.2708129515250524

2001/? [11:48<00:00, 3.01it/s]
Среднее значение функции потерь на обучении 0.716283163626393
Среднее значение функции потерь на валидации 0.268450199564298
Новая лучшая модель!

2001/? [11:44<00:00, 3.02it/s]
Среднее значение функции потерь на обучении 0.7147186489059948
Среднее значение функции потерь на валидации 0.2660525545477867
Новая лучшая модель!

2001/? [11:42<00:00, 2.87it/s]
Среднее значение функции потерь на обучении 0.7126921191923264
Среднее значение функции потерь на валидации 0.263121996819973
Новая лучшая модель!

2001/? [11:42<00:00, 3.00it/s]
Среднее значение функции потерь на обучении 0.7097944344418576
Среднее значение функции потерь на валидации 0.2610153297583262
Новая лучшая модель!
```

Around 32 epoches alpha = 0. Then alpha = 1e-4